# Zero-Cost Proxy Evaluation  w/ minibatch

Testing zero-cost proxies with zone-agnostic behavior policy model

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr

In [2]:
log_data = pd.read_csv("../data/rule_based_log_data/0_cleaned_log.csv")
with open("../data/rule_based_log_data/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
with open("../data/invalid_policy_list.json") as f:
    invalid_policies = json.load(f)["invalid_policies"]

## 30 Day MiniBatch

In [3]:
num_ts_per_day = 4 * 24
num_days = 30
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [4]:
policy_list = sorted(list(glob.glob(f"../policy_library/**.pth")))

### 1. GradNorm

In [5]:
from zero_cost_proxies.grad_norm import GradNorm
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        gn = GradNorm(agent, behavior_model)
        if policy not in invalid_policies:
            norm = gn.get_grad_norm(ope_data)
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = norm

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [7]:
spearman_corr

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.5542986268153618, pvalue=2.0152859447344407e-46),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.6055320455715625, pvalue=3.1586415731606343e-57),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.6020077561584476, pvalue=2.0513169884514363e-56),
 'Core_bottom': SpearmanrResult(correlation=-0.5054980182218064, pvalue=1.3570434889641183e-37),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.5581392053438957, pvalue=4.2975391166151174e-47),
 'Core_top': SpearmanrResult(correlation=-0.5270601334613021, pvalue=2.769141655022239e-41),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.5559879266265633, pvalue=1.1355647085317673e-46),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.5357703475952144, pvalue=7.497300095924579e-43),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.5527398258873174, pvalue=4.8591214781307e-46),
 'Core_mid': SpearmanrResult(correlation=-0.5254026550131286, pvalue=5.438648360275325

In [8]:
with open("grad_norm_ipw_spearman_corrs_03_07_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Synflow

In [5]:
from zero_cost_proxies.synflow import SynFlow
policy_scores_synflow = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        sf = SynFlow(agent)
        if policy not in invalid_policies:
            synflow = sf.get_synflow()
        else:
            continue
        if policy not in policy_scores_synflow:
            policy_scores_synflow[policy] = {}
        if zone not in policy_scores_synflow[policy]:
            policy_scores_synflow[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [6]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_synflow = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_synflow[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_synflow[zone] = correlation

In [7]:
spearman_corr_synflow

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=0.021068979260633003, pvalue=0.6188194248841729),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.020532605678892247, pvalue=0.6280874619763966),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.0055637535757565755, pvalue=0.8955764421819662),
 'Core_bottom': SpearmanrResult(correlation=0.0030009562830268003, pvalue=0.9435621135883738),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.00791708916240049, pvalue=0.851841679046524),
 'Core_top': SpearmanrResult(correlation=0.042295051784924244, pvalue=0.318183427486605),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=0.02266644957942782, pvalue=0.5928047062533964),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.03253327068030743, pvalue=0.4426824092941606),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.09214570126047994, pvalue=0.02937797870997663),
 'Core_mid': SpearmanrResult(correlation=0.024578740839055913, pvalue=0.5619777235555059),
 'Perimeter_mid_

In [11]:
with open("data/synflow_raw_scores_16_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_synflow, f)

### 3. Jacobian Covariance

In [16]:
from zero_cost_proxies.jacob_cov import JacobianCovariance
policy_scores_jc = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    states = []
    for i, row in ope_data.iterrows():
        state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                      "zone_humidity", "zone_temp", "zone_occupancy"]
        state = [row[var] for var in state_vars]
        states.append(state)
    states = torch.Tensor(states)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        jc = JacobianCovariance(agent)
        if policy not in invalid_policies:
            synflow = jc.compute_jacob_cov(states)
        else:
            continue
        if policy not in policy_scores_jc:
            policy_scores_jc[policy] = {}
        if zone not in policy_scores_jc[policy]:
            policy_scores_jc[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [17]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_jc = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_jc[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_jc[zone] = correlation

In [18]:
spearman_corr_jc

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.0174045861355571, pvalue=0.6810882865445418),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=0.03315217540648898, pvalue=0.4340480497135204),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.016075928322740022, pvalue=0.7044949564062843),
 'Core_bottom': SpearmanrResult(correlation=0.018762176807938243, pvalue=0.6580235886362641),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.027275225399756527, pvalue=0.519867729167059),
 'Core_top': SpearmanrResult(correlation=-0.0074366807264433, pvalue=0.8607373417337911),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.0029774613983166117, pvalue=0.9440032462151668),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.00013238199659805058, pvalue=0.9975082701419334),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=0.008598440818995957, pvalue=0.8392576078923194),
 'Core_mid': SpearmanrResult(correlation=-0.026008219087738964, pvalue=0.5394503503388692),
 'Perimeter_mi

In [20]:
with open("data/jacob_cov_spearman_corr_16_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_jc, f)